In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

set dictionary for data collecting

In [ ]:
head_coaches_of_the_year_dict = {
 'Year': [],
 'Coach': [],
 'Coach URL': [],
 'Team': [],
 'W-L': [],
 'Playoffs W-L': [],
 'Career W-L': [],
 'Exp [Years]': []
}

collect Coach of The Year Award data from site

In [ ]:
head_coaches_of_the_year_url = 'http://www.espn.com/nba/history/awards/_/id/34'
  
page = requests.get(head_coaches_of_the_year_url)
soup = BeautifulSoup(page.text, 'html.parser')
tables = soup.find_all('table')
rows = tables[0].findAll('tr')[2:]

set data into dictionary and then to dataframe

In [ ]:
for row in rows:
    cols = row.findAll('td')

    head_coaches_of_the_year_dict['Year'].append(cols[0].string)
    head_coaches_of_the_year_dict['Coach'].append(cols[1].string)
    head_coaches_of_the_year_dict['Coach URL'].append(cols[1].find_all('a', href=True)[0]['href'])
    head_coaches_of_the_year_dict['Team'].append(cols[2].string)
    head_coaches_of_the_year_dict['W-L'].append(cols[3].string)
    head_coaches_of_the_year_dict['Playoffs W-L'].append(cols[4].string)
    head_coaches_of_the_year_dict['Career W-L'].append(cols[5].string)
    head_coaches_of_the_year_dict['Exp [Years]'].append(cols[6].string.split()[0])
    
df_head_coaches_of_the_year = pd.DataFrame(head_coaches_of_the_year_dict)

In [ ]:
df_head_coaches_of_the_year['W'] = df_head_coaches_of_the_year['W-L'].apply(lambda x: x.split('-')[0])
df_head_coaches_of_the_year['L'] = df_head_coaches_of_the_year['W-L'].apply(lambda x: x.split('-')[1])
df_head_coaches_of_the_year['Playoffs W'] = df_head_coaches_of_the_year['Playoffs W-L'].apply(lambda x: x.split('-')[0] if x is not None else '')
df_head_coaches_of_the_year['Playoffs L'] = df_head_coaches_of_the_year['Playoffs W-L'].apply(lambda x: x.split('-')[1] if x is not None else '')
df_head_coaches_of_the_year['Career W'] = df_head_coaches_of_the_year['Career W-L'].apply(lambda x: x.split('-')[0] if x is not None else '')
df_head_coaches_of_the_year['Career L'] = df_head_coaches_of_the_year['Career W-L'].apply(lambda x: x.split('-')[1] if x is not None else '')

Add Coaches Details from Site

In [ ]:
def get_coach_details(i_coach_url):  

    output_dict = {}

    head_coach_data_url = 'http://www.espn.com' + i_coach_url
    page = requests.get(head_coach_data_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    tables = soup.find_all('table')
    rows = tables[0].findAll('tr')[3:]

    output_dict['Born'] = rows[0].text[4:]
    output_dict['Birthplace'] = rows[1].text[10:]
    output_dict['Experience'] = rows[2].text[10:].split()[0]
    output_dict['College'] = rows[3].text[7:].replace('--','')
    output_dict['Overall Record'] = rows[4].text[6:]

    return output_dict
    

In [ ]:
df_head_coaches_of_the_year['Born'] = None
df_head_coaches_of_the_year['Birthplace'] = None
df_head_coaches_of_the_year['College'] = None
df_head_coaches_of_the_year['Overall Record'] = None

for index, row in df_head_coaches_of_the_year.iterrows():
    coach_dict_details = get_coach_details(row['Coach URL'])
    
    df_head_coaches_of_the_year.iloc[index]['Born'] = coach_dict_details['Born']
    df_head_coaches_of_the_year.iloc[index]['Birthplace'] = coach_dict_details['Birthplace']
    df_head_coaches_of_the_year.iloc[index]['College'] = coach_dict_details['College']
    df_head_coaches_of_the_year.iloc[index]['Overall Record'] = coach_dict_details['Overall Record']

df_head_coaches_of_the_year.drop('Coach URL', axis=1, inplace=True)

save DataFrame to csv file

In [ ]:
df_head_coaches_of_the_year.to_csv('NBA_coach_of_the_year.csv', index=False)